# Práctica Guiada de Laboratorio 3: Implementación del juego inteligente *Michi* en Python - Parte 2

# Curso "Inteligencia Artificial" Semestre 2025-1 - Sección 1 - EPISW

### Prof. Rolando A. Maguiña Pérez
### Abril 2025

## Introducción
En la primera parte de la Práctica Guiada  se presentó la construcción en Python de la plataforma de juego para el ampliamente conocido y popular **Tres en raya** o **Michi**; no solamente se construyó la interfaz del juego sino que, también, el programa permitía efectuar las jugadas del humano. Dicha implementación se hizo a nivel de consola únicamente. Asimismo, en dicha primera parte, se  implementó una estrategia de juego para la máquina que determinaba sus jugadas en dicho juego inteligente; en ese caso, se trató de la estrategia aleatoria.

En esta segunda parte de la Práctica Guiada de Laboratorio 3, se presenta la implementación de la estrategia de búsqueda denominada **Primero el Mejor** que determinará las jugadas de la máquina. Con esa finalidad, se usará como base el programa  construido en la segunda parte de la Práctica Guiada anterior y se insertarán en el código respectivo, las funciones adecuadas que implementen la estrategia mencionada. Asimismo, el programa indicará la estrategia usada para efectuar dichos movimientos.

## Estrategia Primero el Mejor
## Implementación de la estrategia Primero el Mejor
La implementación de esta estrategia de búsqueda para las jugadas de la máquina, requiere básicamente de tres pasos:
- Generar los sucesores de un nodo.
- Definir una función de evaluación ad hoc y aplicarla a los sucesores de dicho nodo.
- Seleccionar el sucesor con mejor valor de evaluación.

### Generación de sucesores
Definiremos una función para la generación de los sucesores que llamaremos `genmov(.)`. Antes,  debemos modificar la denominada `efectuar_movimiento(.)`; en la nueva versión de esta función, se crea previamente una copia del tablero, luego se efectúa el movimiento (coloca en la posición *pos* un aspa 'X' si es el jugador computadora o un círculo 'O' si es el jugador oponente).

La nueva versión es:

#### Función *efectuar_movimiento(.)*

In [31]:
def efectuar_movimiento(jugador, pos, tablero):
    tablero=tablero.copy()
    pos=pos
    tablero[pos]=jugador
    return tablero

Para comprobar el comportamiento de la función así definida,  luego de interpretarla la llamamos para marcar un aspa en la posición 3 del tablero:

In [32]:
efectuar_movimiento(1, 8, [0,0,0,0,0,0,0,0,0])

[0, 0, 0, 0, 0, 0, 0, 0, 1]

#### Función *genmov(.)*

La función `genmov(.)` toma un tablero y un jugador, y genera sus sucesores, es decir, todos los posibles movimientos que un jugador podría realizar. Lo hace mediante el llamado de la función `efectuar_movimiento()`, pero verificando antes si la posición del movimiento corresponde a un casillero vacío; si corresponde, se efectúa el movimiento.

In [33]:
def genmov(tablero, jugador):
    lista=[]
    for i in range(0,9):
        if (tablero[i]==0):
            sucesor=efectuar_movimiento(jugador,i,tablero)
            lista.append(sucesor)
        else:
            tablero[i]=tablero[i]
    return lista

Verificando la función con un tablero sin jugada alguna, para el jugador humano.

In [34]:
genmov([0,0,0,0,0,0,0,0,0],1)

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1]]

Verificando ahora la función con un tablero en el que se han realizado varias jugadas, para el jugador humano:

In [35]:
genmov([0,1,0,10,0,10,10,1,0],1)

[[1, 1, 0, 10, 0, 10, 10, 1, 0],
 [0, 1, 1, 10, 0, 10, 10, 1, 0],
 [0, 1, 0, 10, 1, 10, 10, 1, 0],
 [0, 1, 0, 10, 0, 10, 10, 1, 1]]

Se observa que al llamársela con un tablero con varias jugadas realizadas y jugando el humano, la función generó los sucesores posibles: posiciones 0, 2, 4 y 8.

### Función de evaluación
Se usará la FUNCIÓN DE EVALUACIÓN ESTATICA que  plantean Nilsson/Norvig en su libro  y que se presentó en clase de teoría:

- Si *p* no es una posición ganadora para ningún jugador:

       f(p) = (Nº filas, columnas y diagonales completas aun libres para MAX) - (Nº filas, columnas y diagonales completas aun libres para MIN)
- Si *p* es una posición ganadora para MAX:

       f(p) = +inf (donde +inf indica un número positivo muy grande)
- Si *p* es una posición ganadora para MAX:

       f(p) = -inf (donde -inf indica un número negativo muy grande)
       
#### Función *feval(.)*
En este programa se implementará mediante la función `feval(.)`. Esta, a su vez,  se vale de las funciones `ganador_pc(.)` y `ganador_oponente(.)`, que calculan f(p) cuando *p*  corresponde a una posición ganadora de uno de los jugadores. Usa también la función denominada `lineas-abiertas(.)` que calcula f(p) cuando *p* no corresponde a una posición ganadora para jugador alguno; en ese caso, calcula la diferencia entre líneas abiertas para MAX menos líneas abiertas para MIN.

In [36]:
# Fc de evaluacion;
def feval(tablero):
    if(ganador_pc(tablero)==True):
        res = 10000
        return res
    elif(ganador_oponente(tablero)==True):
        res = -10000
        return res
    else:
        return lineas_abiertas(tablero)

Definiremos ahora las funciones `ganador_pc(.)`, `ganador_oponente(.)` y `lineas_abiertas(.)`, para poder usar `feval(.)`.

#### Función *ganador_pc(.)*
- La función `ganador_pc(.)` recibe como parámetro un tablero y devuelve True si ha ganado la máquina; la definimos así:

In [37]:
def ganador_pc(tablero):
    sumas = calcula_sumas(tablero)
    if 30 in sumas:
        return True
    else:
        return False

Llamamos a la función para comprobar su funcionamiento; previamente, recordamos las definiciones de las funciones `suma_tripleta(.)` y `calcula_sumas(.)`, y las interpretamos:

In [38]:
tripletas = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]

def suma_tripleta(tablero, tripleta):
    a, b, c = tripleta[0], tripleta[1], tripleta[2]
    return tablero[a] + tablero[b] + tablero[c]

def calcula_sumas(tablero):
    sumas = []
    for x in tripletas:
        z = suma_tripleta(tablero, x)
        sumas.append(z)
    return sumas

In [39]:
ganador_pc([10,10,10,1,0,0,1,0,1])

True

In [40]:
ganador_pc([1,1,10,10,0,0,10,0,1])

False

#### Función *ganador_oponente()*
- Definimos la función `ganador_oponente(.)` que recibe como parámetro un tablero y devuelve True si ha ganado el jugador humano.

In [41]:
def ganador_oponente(tablero):
    sumas = calcula_sumas(tablero)
    if 3 in sumas:
        return True
    else:
        return False

Llamamos a la función para comprobar su funcionamiento:

In [42]:
ganador_oponente([10,10,10,1,0,0,1,0,1])

False

In [43]:
ganador_oponente([1,1,1,10,0,0,10,0,10])

True

#### Función *lineas_abiertas(.)*

Para las posiciones que no corresponden a la victoria de uno de los jugadores, esta función hace lo sgte::
- Para cada elemento de *sumas*  (recordar que *sumas* contiene una lista con los valores de la suma de los elementos de las 8 tripletas)
    - La condicional IF verifica los valores de c/u de ellos:
        - Si es 0: suma 1 a abiertasX y a abiertasO
        - Si es 1: suma 1 a abiertasO
        - Si es 2: suma 1 a abiertasO
        - Si es 10: suma 1 a abiertasX
        - Si es 20: suma 1 a abiertasX
- Devuelve (abiertasX – abiertasO)

In [44]:
def lineas_abiertas(tab):
    sumas = calcula_sumas(tab)
    abiertasX = 0
    abiertasO = 0
    for s in sumas:
        if(s==0):
            abiertasX = abiertasX + 1
            abiertasO = abiertasO + 1
        elif(s==1):
            abiertasO = abiertasO + 1
        elif(s==2):
            abiertasO = abiertasO + 1
        elif(s==10):
            abiertasX = abiertasX + 1
        elif(s==20):
            abiertasX = abiertasX + 1

    res = abiertasX - abiertasO
    return res

Llamamos a la función para comprobar su funcionamiento:

In [45]:
lineas_abiertas([0,1,0,0,10,0,0,0,0])

2

In [46]:
lineas_abiertas([10,1,0,1,0,10,0,0,0])

1

In [47]:
lineas_abiertas([0,1,0,0,0,0,0,10,0])

0

- En  las tres veces que se llamó a la función `lineas_abiertas(.)`, pasándosele como parámetro un tablero, devolvió el valor correcto.

Ahora que se han definido las funciones `ganador_pc()`, `ganador_oponente()` y `lineas_abiertas()`, llamamos a `feval()` para comprobar su funcionamiento:

In [48]:
feval([1,1,1,10,10,0,0,0,10])

-10000

In [49]:
feval([1,1,0,0,1,0,10,10,10])

10000

Para comprobar que llama correctamente a `lineas_abiertas(.)`, llamamos nuevamente a la función `feval(.)` pasándole como argumento un tablero en el que no hay un ganador.

In [50]:
feval([1,1,0,0,10,0,0,0,10])

2

In [51]:
feval([0,1,0,0,10,0,0,0,0])

2

Se comprobó que se está comportando correctamente.

### Selección de estrategia a usar por la máquina
A la función `escoger_estrategia_pc()`, definida en la parte 2 de la Práctica anterior, sólo se le cambiará la estrategia para las jugadas de la máquina; ahora se usará la `estrategia_primero_el_mejor()` en lugar de `estrategia_aleatoria()`. Enseguida, se presenta su definición.

#### Función *escoger_estrategia_pc(.)*

In [52]:
def escoger_estrategia_pc(tablero):
    mejor_movimiento = estrategia_primero_el_mejor(tablero)
    return mejor_movimiento

#### Función *estrategia_primero_el_mejor(.)*
Esta función llama a `seleccion-primero-el-mejor()` con dos argumentos, *tablero* y  *pc*. Devuelve una lista con la posición elegida para marcar un aspa  y un texto descriptivo.

In [53]:
def estrategia_primero_el_mejor(tablero):

    valor=seleccion_primero_el_mejor(tablero,pc)

    tupla= valor,"Movimiento por Primero el Mejor"

    return tupla

#### Selección del mejor sucesor
Se implementará mediante las funciones `sucesores_con_evaluacion(.)` y `seleccion_primero_el_mejor(.)`. La primera devuelve una lista de sucesores con su respectivo valor de evaluación, y la segunda, selecciona el mejor sucesor.

####  Función *sucesores_con_evaluacion(.)*
Esta función recibe como parámetros un tablero y una lista  *sucesores*  conteniendo justamente los tableros sucesores. Para calcular el valor de evaluación de los sucesores aplica `feval(.)` a cada uno de ellos, luego junta en una nueva lista cada sucesor con el valor obtenido de esa manera.

In [54]:
def sucesores_con_evaluacion(tablero, sucesores):
    sucesores_peso = []
    for n in sucesores:
        a = feval(n)
        lista = [n,a]
        sucesores_peso.append(lista)
    return sucesores_peso

Llamamos a la función para comprobar su funcionamiento; antes, definimos un tablero para enseguida generar sus sucesores.

In [55]:
tablero = [0,1,0,0,0,0,0,10,0]
sucesores = genmov(tablero,1)
#sucesores
d = sucesores_con_evaluacion(tablero,sucesores)
d

[[[1, 1, 0, 0, 0, 0, 0, 10, 0], -2],
 [[0, 1, 1, 0, 0, 0, 0, 10, 0], -2],
 [[0, 1, 0, 1, 0, 0, 0, 10, 0], -2],
 [[0, 1, 0, 0, 1, 0, 0, 10, 0], -3],
 [[0, 1, 0, 0, 0, 1, 0, 10, 0], -2],
 [[0, 1, 0, 0, 0, 0, 1, 10, 0], -3],
 [[0, 1, 0, 0, 0, 0, 0, 10, 1], -3]]

#### Función *selección_primero_el_mejor()*
En esta función se genera la lista de sucesores con sus respectivos valores de evaluación y luego de haberlos ordenado, se obtiene la posición donde se marcó el aspa 'X' en el mejor tablero; busca mediante el bucle *while* dicha posición *i* en el tablero.

In [56]:
from operator import itemgetter

def seleccion_primero_el_mejor(tablero,jugador):
    sucesores = genmov(tablero,jugador)
    sucesores_peso = sucesores_con_evaluacion(tablero,sucesores)
    sucesores_ordenados = sorted(sucesores_peso, key=itemgetter(1)).pop()
    z = sucesores_ordenados[0]
    # Obtiene el índice donde se marco el aspa
    i = 0
    while (z[i]==tablero[i]):
        i = i + 1
    return i

Llamamos a la función para comprobar su funcionamiento:

In [57]:
seleccion_primero_el_mejor([0,0,0,0,0,0,0,0,0],10)

4

In [58]:
seleccion_primero_el_mejor([0,1,0,0,0,0,0,10,0],1)

5

In [59]:
import random

#variables para PC y Oponente
pc=10
oponente=1

#tablero=[0,1,1,0,10,0,0,10,1]
#crea tablero nuevo
def crear_tablero():
    tablero=[0,0,0,0,0,0,0,0,0]
    #print(tablero)
    return tablero


#Convierte valores de 1 -> O e 10 -> X
def convertir_a_letras(v):
    if (v == 1):
        return 'O'
    elif (v == 10):
        return 'X'
    elif (v == 0):
        return " "

def imprimir_fila(x ,y ,z):
     print("    {0}  |  {1}  |  {2} ".format(convertir_a_letras(x),convertir_a_letras(y),convertir_a_letras(z)))




def imprimir_tablero(tablero):
    print("\n")
    imprimir_fila(tablero[0],tablero[1],tablero[2])
    print('   ----------------')
    imprimir_fila(tablero[3],tablero[4],tablero[5])
    print('   ----------------')
    imprimir_fila(tablero[6],tablero[7],tablero[8])
    print("\n")


def efectuar_movimiento(jugador, pos, tablero):
    tablero=tablero.copy()
    pos=pos
    tablero[pos]=jugador
    return tablero

#tripletas = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 4, 7], [2, 5, 8], [3, 6, 9], [1, 5, 9], [3, 5, 7]]
#tripleta1, tripleta2, tripleta3, tripleta4, tripleta5, tripleta6, tripleta7, tripleta8 = [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 4, 7], [2, 5, 8], [3, 6, 9], [1, 5, 9], [3, 5, 7]
#tripletas = [tripleta1, tripleta2, tripleta3, tripleta4, tripleta5, tripleta6, tripleta7, tripleta8]
tripletas = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]

# Suma los valores asociados a las posiciones de *tripleta*
def suma_tripleta(tablero, tripleta):
    a, b, c = tripleta[0], tripleta[1], tripleta[2]
    # print(a,b,c); se mando imprimir para verificacion
    return tablero[a] + tablero[b] + tablero[c]


# Fc iterativa que obtiene las sumas de las *tripletas* de un tablero
def calcula_sumas(tablero):
    # global sumas
    sumas = []
    for x in tripletas:
        z = suma_tripleta(tablero, x)
        sumas.append(z)
    return sumas

# Determina si hay un ganador en la partida
def ganador_p(tablero):
    sumas = calcula_sumas(tablero)
    gana_op=False
    gana_pc=False


    if 3 in sumas:
        gana_op=True
            #print(ganador)

    if 30 in sumas:
        gana_pc=True
            #print(ganador)

    if gana_op==True and gana_pc==False:
        ganador=gana_op
    elif gana_op==False and gana_pc==True:
        ganador=gana_pc
    else:
        ganador=False
    return ganador

def inicio():

    print("""\n
          ------ JUEGO DEL MICHI -----
          Le gustaria comenzar el juego?
          [1] Si
          [2] NO
          [3] Salir
    """)
    opcion=int(input("Seleccione la opcion: \n"))

    if opcion == 1:
        print('oponente inicia el juego')
        movimiento_oponente(crear_tablero())
    elif opcion ==2:
        #print ('maquina inicia juego')
        movimiento_pc(crear_tablero())
    elif opcion==3:
        print("\n")
        print("Saliendo ...")


def validar_movimiento(tablero):
    global cent
    print(tablero)
    pos=int(input("Ingrese tu movimiento: "))
    if(tablero[pos]==0 ):
        pos=pos
        cent = pos
    else:
        #pos=int(input("Ingrese un movimiento valido: "))
        print("casilla ocupada")
        validar_movimiento(tablero)

    return pos


def movimiento_oponente(tablero):

    posi=(validar_movimiento(tablero))

    nuevo_tablero=(efectuar_movimiento(oponente,cent,tablero))

    print("Mi movimiento: ",posi)
    imprimir_tablero(nuevo_tablero)

    if ganador_p(nuevo_tablero) == True:
        print("    Gana Oponente")
        inicio()
    elif tablero_lleno_p == True:
        print("    Empate")
    else:
        movimiento_pc(nuevo_tablero)


def tablero_lleno_p(tablero): #Retorna True si no hay celdas vacias

    for i in tablero:
        #print(len(tablero))
        #print(tablero[i])
        if tablero[i] == 0:
            print(tablero[i])
            return False
        else:
            return True



def movimiento_pc(tablero):
    print("ahora le toca jugar a la PC")
    mejor_movimiento = escoger_estrategia_pc(tablero)
    pos = mejor_movimiento[0]
    estrategia = mejor_movimiento[1]

    nuevo_tablero = (efectuar_movimiento(pc, pos, tablero))

    print("Movimiento: " ,pos )
    print("Estrategia: ", estrategia)

    imprimir_tablero(nuevo_tablero)

    if ganador_p(nuevo_tablero) == True:
        print("    Gana PC")
        inicio()
    elif tablero_lleno_p == True:
        print("    Empate")
    else:
        movimiento_oponente(nuevo_tablero)

    #return tablero


def genmov(tablero,jugador):
    lista=[]
    for i in range(0,9):
        if (tablero[i]==0):
            sucesor=efectuar_movimiento(jugador,i,tablero)
            lista.append(sucesor)
        else:
            tablero[i]=tablero[i]
    return lista


# Fc de evaluacion;
def feval(tablero):
    if(ganador_pc(tablero)==True):
        res = 10000
        return res
    elif(ganador_oponente(tablero)==True):
        res = -10000
        return res
    else:
        return lineas_abiertas(tablero)



def ganador_pc(tablero):
    sumas = calcula_sumas(tablero)
    if 30 in sumas:
        return True
    else:
        return False


def ganador_oponente(tablero):
    sumas = calcula_sumas(tablero)
    if 3 in sumas:
        return True
    else:
        return False

def lineas_abiertas(pos):
    sumas = calcula_sumas(pos)
    abiertasX = 0
    abiertasO = 0
    for s in sumas:
        if(s==0):
            abiertasX = abiertasX + 1
            abiertasO = abiertasO + 1
        elif(s==1):
            abiertasO = abiertasO + 1
        elif(s==2):
            abiertasO = abiertasO + 1
        elif(s==10):
            abiertasX = abiertasX + 1
        elif(s==20):
            abiertasX = abiertasX + 1

    res = abiertasX - abiertasO
    return res


def escoger_estrategia_pc(tablero):
    mejor_movimiento = estrategia_primero_el_mejor(tablero)
    return mejor_movimiento

def estrategia_primero_el_mejor(tablero):

    valor=seleccion_primero_el_mejor(tablero,pc)

    tupla= valor,"Movimiento por Primero el Mejor"

    return tupla


def sucesores_con_evaluacion(tablero, sucesores):
    sucesores_peso = []
    for n in sucesores:
        a = feval(n)
        lista = [n,a]
        sucesores_peso.append(lista)
    return sucesores_peso

from operator import itemgetter

def seleccion_primero_el_mejor(tablero,jugador):
    sucesores = genmov(tablero,jugador)
    sucesores_peso = sucesores_con_evaluacion(tablero,sucesores)
    sucesores_ordenados = sorted(sucesores_peso, key=itemgetter(1)).pop()
    z = sucesores_ordenados[0]
    i = 0
    while (z[i]==tablero[i]):
        i = i + 1
    return i


inicio()





          ------ JUEGO DEL MICHI -----
          Le gustaria comenzar el juego?
          [1] Si
          [2] NO
          [3] Salir
    


ValueError: invalid literal for int() with base 10: ''

## Ejercicios
Respecto de las funciones definidas en la implementación de la estrategia primero el mejor para las jugadas de la máquina, hacer:

### Ejercicio 1
- Optimizar el código presentado.

### Ejercicio 2
- Implementar la estrategia de búsqueda minimax para las jugadas de la máquina. Presentar el código que permite dicha implementación y las instrucciones para jugar.

Solución del ejercicio 2

- Se define la función estrategia_minimax como la función a ejecutar en escoger_estrategia_pc.

In [ ]:
def escoger_estrategia_pc(tablero):
    mejor_movimiento = estrategia_minimax(tablero)
    return mejor_movimiento

def estrategia_minimax(tablero):
    pos = mejor_movimiento_minimax(tablero)
    return pos, "Movimiento por Minimax"

- mejor_movimiento_minimax retorna la posición con la mayor evaluación desde la perspectiva del PC(MAX) aplicando el algoritmo Minmax.

In [ ]:
def mejor_movimiento_minimax(tablero):
    valor,pos=max_minimax(tablero)
    return pos

- max_minimax simula el turno del PC, que busca maximizar su ganancia. Explora todas las jugadas disponibles y elige la de mayor valor evaluado.

In [ ]:
def max_minimax(tablero):
  if es_tablero_terminal(tablero):    
        return feval_minmax(tablero)
  
  mejor_valor = float('-inf')
  mejor_pos = None
  for i in range(9):
      if tablero[i] == 0:
          valor,pos = min_minimax(efectuar_movimiento(pc, i, tablero))
          if valor > mejor_valor:
              mejor_valor = valor
              mejor_pos = i

  return mejor_valor,mejor_pos

- min_minimax simula el turno del oponente, que busca minimizar la ganancia del PC. Evalúa posibles respuestas y selecciona la jugada de menor valor.

In [ ]:
def min_minimax(tablero):
  if es_tablero_terminal(tablero):    
        return feval_minmax(tablero)
  
  mejor_valor = float('+inf')
  mejor_pos = None
  for i in range(9):
      if tablero[i] == 0:
          valor,pos = max_minimax(efectuar_movimiento(oponente, i, tablero))
          if valor < mejor_valor:
              mejor_valor = valor
              mejor_pos = i

  return mejor_valor,mejor_pos

- Verifica si el juego ha llegado a un estado terminal. Retorna True si hay un ganador o el tablero está lleno.

In [ ]:
def es_tablero_terminal(tablero):
    return ganador_pc(tablero) or ganador_oponente(tablero) or tablero_lleno_p(tablero)

- Función de evaluación aplicada a los nodos o estados finales de los tableros. Devuelve 1 si gana el PC, -1 si gana el oponente, o 0 en caso de empate.

In [ ]:
def feval_minmax(tablero):
  if ganador_pc(tablero):
    return 1
  elif ganador_oponente(tablero):
    return -1
  elif(tablero_lleno_p(tablero)): 
    return 0 

Código completo

In [5]:
import random

#variables para PC y Oponente
pc=10
oponente=1

#tablero=[0,1,1,0,10,0,0,10,1]
#crea tablero nuevo
def crear_tablero():
    tablero=[0,0,0,0,0,0,0,0,0]
    #print(tablero)
    return tablero


#Convierte valores de 1 -> O e 10 -> X
def convertir_a_letras(v):
    if (v == 1):
        return 'O'
    elif (v == 10):
        return 'X'
    elif (v == 0):
        return " "

def imprimir_fila(x ,y ,z):
     print("    {0}  |  {1}  |  {2} ".format(convertir_a_letras(x),convertir_a_letras(y),convertir_a_letras(z)))


def imprimir_tablero(tablero):
    print("\n")
    imprimir_fila(tablero[0],tablero[1],tablero[2])
    print('   ----------------')
    imprimir_fila(tablero[3],tablero[4],tablero[5])
    print('   ----------------')
    imprimir_fila(tablero[6],tablero[7],tablero[8])
    print("\n")


def efectuar_movimiento(jugador, pos, tablero):
    tablero=tablero.copy()
    pos=pos
    tablero[pos]=jugador
    return tablero

#tripletas = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 4, 7], [2, 5, 8], [3, 6, 9], [1, 5, 9], [3, 5, 7]]
#tripleta1, tripleta2, tripleta3, tripleta4, tripleta5, tripleta6, tripleta7, tripleta8 = [1, 2, 3], [4, 5, 6], [7, 8, 9], [1, 4, 7], [2, 5, 8], [3, 6, 9], [1, 5, 9], [3, 5, 7]
#tripletas = [tripleta1, tripleta2, tripleta3, tripleta4, tripleta5, tripleta6, tripleta7, tripleta8]
tripletas = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]

# Suma los valores asociados a las posiciones de *tripleta*
def suma_tripleta(tablero, tripleta):
    a, b, c = tripleta[0], tripleta[1], tripleta[2]
    # print(a,b,c); se mando imprimir para verificacion
    return tablero[a] + tablero[b] + tablero[c]


# Fc iterativa que obtiene las sumas de las *tripletas* de un tablero
def calcula_sumas(tablero):
    # global sumas
    sumas = []
    for x in tripletas:
        z = suma_tripleta(tablero, x)
        sumas.append(z)
    return sumas

# Determina si hay un ganador en la partida
def ganador_p(tablero):
    sumas = calcula_sumas(tablero)
    gana_op=False
    gana_pc=False


    if 3 in sumas:
        gana_op=True
            #print(ganador)

    if 30 in sumas:
        gana_pc=True
            #print(ganador)

    if gana_op==True and gana_pc==False:
        ganador=gana_op
    elif gana_op==False and gana_pc==True:
        ganador=gana_pc
    else:
        ganador=False
    return ganador

def inicio():

    print("""\n
          ------ JUEGO DEL MICHI -----
          Le gustaria comenzar el juego?
          [1] Si
          [2] NO
          [3] Salir
    """)
    opcion=int(input("Seleccione la opcion: \n"))

    if opcion == 1:
        print('oponente inicia el juego')
        movimiento_oponente(crear_tablero())
    elif opcion ==2:
        #print ('maquina inicia juego')
        movimiento_pc(crear_tablero())
    elif opcion==3:
        print("\n")
        print("Saliendo ...")


def validar_movimiento(tablero):
    global cent
    print(tablero)
    pos=int(input("Ingrese tu movimiento: "))
    if(tablero[pos]==0 ):
        pos=pos
        cent = pos
    else:
        #pos=int(input("Ingrese un movimiento valido: "))
        print("casilla ocupada")
        validar_movimiento(tablero)

    return pos


def movimiento_oponente(tablero):

    posi=(validar_movimiento(tablero))

    nuevo_tablero=(efectuar_movimiento(oponente,cent,tablero))

    print("Mi movimiento: ",posi)
    imprimir_tablero(nuevo_tablero)

    if ganador_p(nuevo_tablero) == True:
        print("    Gana Oponente")
        inicio()
    elif tablero_lleno_p(nuevo_tablero) == True:
        print("    Empate")
    else:
        movimiento_pc(nuevo_tablero)


def tablero_lleno_p(tablero):
    for i in tablero:
        if i == 0:
            return False
    return True

def movimiento_pc(tablero):
    print("ahora le toca jugar a la PC")
    mejor_movimiento = escoger_estrategia_pc(tablero)
    pos = mejor_movimiento[0]
    estrategia = mejor_movimiento[1]

    nuevo_tablero = (efectuar_movimiento(pc, pos, tablero))

    print("Movimiento: " ,pos )
    print("Estrategia: ", estrategia)

    imprimir_tablero(nuevo_tablero)

    if ganador_p(nuevo_tablero) == True:
        print("    Gana PC")
        inicio()
    elif tablero_lleno_p(nuevo_tablero) == True:
        print("    Empate")
    else:
        movimiento_oponente(nuevo_tablero)

    #return tablero

def ganador_pc(tablero):
    sumas = calcula_sumas(tablero)
    if 30 in sumas:
        return True
    else:
        return False


def ganador_oponente(tablero):
    sumas = calcula_sumas(tablero)
    if 3 in sumas:
        return True
    else:
        return False
    
def feval_minmax(tablero):
  if ganador_pc(tablero):
    return 1
  elif ganador_oponente(tablero):
    return -1
  elif(tablero_lleno_p(tablero)): 
    return 0 
    
def es_tablero_terminal(tablero):
    return ganador_pc(tablero) or ganador_oponente(tablero) or tablero_lleno_p(tablero)

def min_minimax(tablero):
  if es_tablero_terminal(tablero):    
        return feval_minmax(tablero), None
  
  mejor_valor = float('+inf')
  mejor_pos = None
  for i in range(9):
      if tablero[i] == 0:
          valor,pos = max_minimax(efectuar_movimiento(oponente, i, tablero))
          if valor < mejor_valor:
              mejor_valor = valor
              mejor_pos = i

  return mejor_valor,mejor_pos

def max_minimax(tablero):
  if es_tablero_terminal(tablero):    
        return feval_minmax(tablero), None
  
  mejor_valor = float('-inf')
  mejor_pos = None
  for i in range(9):
      if tablero[i] == 0:
          valor,pos = min_minimax(efectuar_movimiento(pc, i, tablero))
          if valor > mejor_valor:
              mejor_valor = valor
              mejor_pos = i

  return mejor_valor,mejor_pos
    
def mejor_movimiento_minimax(tablero):
    valor,pos=max_minimax(tablero)
    return pos
    
def estrategia_minimax(tablero):
    pos = mejor_movimiento_minimax(tablero)
    return pos, "Movimiento por Minimax"

def escoger_estrategia_pc(tablero):
    mejor_movimiento = estrategia_minimax(tablero)
    return mejor_movimiento

inicio()



          ------ JUEGO DEL MICHI -----
          Le gustaria comenzar el juego?
          [1] Si
          [2] NO
          [3] Salir
    


ahora le toca jugar a la PC
Movimiento:  0
Estrategia:  Movimiento por Minimax


    X  |     |    
   ----------------
       |     |    
   ----------------
       |     |    


[10, 0, 0, 0, 0, 0, 0, 0, 0]
Mi movimiento:  1


    X  |  O  |    
   ----------------
       |     |    
   ----------------
       |     |    


ahora le toca jugar a la PC
Movimiento:  3
Estrategia:  Movimiento por Minimax


    X  |  O  |    
   ----------------
    X  |     |    
   ----------------
       |     |    


[10, 1, 0, 10, 0, 0, 0, 0, 0]
Mi movimiento:  6


    X  |  O  |    
   ----------------
    X  |     |    
   ----------------
    O  |     |    


ahora le toca jugar a la PC
Movimiento:  4
Estrategia:  Movimiento por Minimax


    X  |  O  |    
   ----------------
    X  |  X  |    
   ----------------
    O  |     |    


[10, 1, 0, 10, 10, 0, 1, 0, 0]
Mi movimiento:  2


    X  |  O  |  O 
   ----------------
    X  |  X  |    
   ----------------
    O  |     |    


ahora le toca

# Instrucciones para el envío de la solución
- La solución de la "Práctica de Laboratorio 3 Parte 2 - IA 2024-1 G1 EPISW" deberá enviarse al correo electrónico rmaguinacursos@gmail.com, hasta las 23:59 h del Miércoles 17 de Abril del 2025, en un archivo con extensión .ipynb.

El documento deberá tener las sgtes características:
- Nombre del archivo:
  solPGL3_Parte2_IA_2024-1_G3_nombre-apellidos-alumno1_...nombre-apellidos-alumno4.ipynb.
- Todas las preguntas de la Práctica deben responderse en este mismo cuaderno computacional interactivo (**Sugerencia**: obtener una copia de este documento y desarrollar en ellas las respectivas soluciones); la solución a cada pregunta debe registrarse en una celda debajo del plnteamiento de la misma, mencionando explícitamente como subtítulo: "Solución del ejercicio n", donde "n" corresponde al número del ejercicio.